In [2]:
import pandas as pd
import numpy as np
import json
import geopandas
import geopy
import requests
from IPython.display import IFrame
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library
from sklearn.cluster import KMeans
import shapely.geometry
import pyproj
import csv
import numbers
import math

ImportError: DLL load failed while importing _network: 找不到指定的模块。

In [3]:
LD_grid = pd.read_csv("grid.csv",engine='python',sep=',')
#print(LD_grid)
LD_grid_venues = pd.read_csv("ALLMEDIA.csv",encoding = "unicode_escape", engine='python',sep=',')

Clustering =  4 # Number of clustering

In [4]:
LD_grid_venues

,grid,latitude,longitude,keywords,Venue Category
0,"16,9",48.893696,2.248569,"alvancewheel, commerci, court, step, save, com...",Travel and Transportation
1,"15,1",48.886973,2.247657,"trailer, kill, eve, announc, concert, suppos, ...",Travel and Transportation
2,"15,1",48.886529,2.247471,"trailer, kill, eve, announc, concert, suppos, ...",Travel and Transportation
3,"2,4",48.889558,2.229056,"trail, ant, room",Travel and Transportation
4,"6,0",48.886266,2.234462,"flight, believ",Travel and Transportation
...,...,...,...,...,...
474,"17,3",48.888402,2.250197,train,Travel and Transportation
475,"7,4",48.889700,2.235707,"tower, tower-1983-1988",Business and_Professional Services
476,"9,8",48.892670,2.239065,event,Arts and Entertainment
477,"9,8",48.892670,2.239065,event,Arts and Entertainment


In [1]:
# create map, LD_tweets['text']
# set location that make Hong Kong at the centre of the web page
# set zoom_start = 12 for better visualization
map = folium.Map(tiles='cartodbpositron', location=[48.89114492, 2.240388888], zoom_start=14.6)

# add markers to the map to show the districts
markers_colors = []
for lat, lon, grd in zip(LD_grid['latitude'], LD_grid['longitude'],LD_grid['grid']):
    label = folium.Popup(str(grd))
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        fill=True,
        fill_color='#ff6951',
        fill_opacity=0.4).add_to(map)
       
map.save('ld_grid.html')
#display(IFrame('ld_grid.html', width = 1000, height = 900))

NameError: name 'folium' is not defined

In [6]:
LD_grid_venues

,grid,latitude,longitude,keywords,Venue Category
0,"16,9",48.893696,2.248569,"alvancewheel, commerci, court, step, save, com...",Travel and Transportation
1,"15,1",48.886973,2.247657,"trailer, kill, eve, announc, concert, suppos, ...",Travel and Transportation
2,"15,1",48.886529,2.247471,"trailer, kill, eve, announc, concert, suppos, ...",Travel and Transportation
3,"2,4",48.889558,2.229056,"trail, ant, room",Travel and Transportation
4,"6,0",48.886266,2.234462,"flight, believ",Travel and Transportation
...,...,...,...,...,...
474,"17,3",48.888402,2.250197,train,Travel and Transportation
475,"7,4",48.889700,2.235707,"tower, tower-1983-1988",Business and_Professional Services
476,"9,8",48.892670,2.239065,event,Arts and Entertainment
477,"9,8",48.892670,2.239065,event,Arts and Entertainment


In [7]:
print(LD_grid_venues.groupby('grid').count())

           latitude  longitude  keywords  Venue Category
grid                                                    
0,11              1          1         1               1
0,2               1          1         1               1
0,8               1          1         1               1
1,12              1          1         1               1
1,3               1          1         1               1
...             ...        ...       ...             ...
9,6              13         13        13              13
9,7               6          6         6               6
9,8              16         16        16              16
9,9               1          1         1               1
Education         2          2         2               2

[138 rows x 4 columns]


In [8]:
# one hot encoding
LD_grid_venues_onehot = pd.get_dummies(LD_grid_venues[['Venue Category']], prefix="", prefix_sep="")

# add District column back to dataframe
LD_grid_venues_onehot['grid'] = LD_grid_venues['grid'] 

# move District column to the first column
fixed_columns = [LD_grid_venues_onehot.columns[-1]] + list(LD_grid_venues_onehot.columns[:-1])
LD_grid_venues_onehot = LD_grid_venues_onehot[fixed_columns]

LD_grid_venues_onehot.head()

,grid,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,Retail,Travel and Transportation
0,"16,9",0,0,0,0,0,0,0,1
1,"15,1",0,0,0,0,0,0,0,1
2,"15,1",0,0,0,0,0,0,0,1
3,"2,4",0,0,0,0,0,0,0,1
4,"6,0",0,0,0,0,0,0,0,1


In [9]:
LD_grid_grouped = LD_grid_venues_onehot.groupby('grid').mean().reset_index()
LD_grid_grouped

,grid,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,Retail,Travel and Transportation
0,"0,11",1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,"0,2",0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,"0,8",1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,"1,12",0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000
4,"1,3",0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
133,"9,6",0.000000,0.000000,0.076923,0.000000,0.0,0.615385,0.076923,0.230769
134,"9,7",0.166667,0.166667,0.000000,0.166667,0.0,0.500000,0.000000,0.000000
135,"9,8",0.500000,0.000000,0.125000,0.062500,0.0,0.062500,0.250000,0.000000
136,"9,9",0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000


In [10]:
# Find out the top 10 restaurant type in the district

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 6

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['grid']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most poi Type'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most poi Type'.format(ind+1))

# create a new dataframe
LD_grid_dish_sorted = pd.DataFrame(columns=columns)
LD_grid_dish_sorted['grid'] = LD_grid_grouped['grid']

for ind in np.arange(LD_grid_grouped.shape[0]):
    LD_grid_dish_sorted.iloc[ind, 1:] = return_most_common_venues(LD_grid_grouped.iloc[ind, :], num_top_venues)

LD_grid_dish_sorted

,grid,1st Most poi Type,2nd Most poi Type,3rd Most poi Type,4th Most poi Type,5th Most poi Type,6th Most poi Type
0,"0,11",Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,Landmarks and Outdoors
1,"0,2",Community and Government,Arts and Entertainment,Business and_Professional Services,Education,Health and Medicine,Landmarks and Outdoors
2,"0,8",Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,Landmarks and Outdoors
3,"1,12",Education,Arts and Entertainment,Business and_Professional Services,Community and Government,Health and Medicine,Landmarks and Outdoors
4,"1,3",Business and_Professional Services,Arts and Entertainment,Community and Government,Education,Health and Medicine,Landmarks and Outdoors
...,...,...,...,...,...,...,...
133,"9,6",Landmarks and Outdoors,Travel and Transportation,Community and Government,Retail,Arts and Entertainment,Business and_Professional Services
134,"9,7",Landmarks and Outdoors,Arts and Entertainment,Business and_Professional Services,Education,Community and Government,Health and Medicine
135,"9,8",Arts and Entertainment,Retail,Community and Government,Education,Landmarks and Outdoors,Business and_Professional Services
136,"9,9",Landmarks and Outdoors,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine


In [11]:
# set number of clusters
kclusters = Clustering

# set the attributes used for clustering
LD_grid_grouped_clustering = LD_grid_grouped.drop('grid', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0)
kmeans.fit(LD_grid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

C:\Users\admin\AppData\Local\Temp\ipykernel_32152\2094119942.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  LD_grid_grouped_clustering = LD_grid_grouped.drop('grid', 1)


array([3, 0, 3, 0, 1, 1, 1, 1, 2, 2])

In [12]:
# Merge the result into the district dataframe
# add clustering labels
LD_grid_dish_sorted['Cluster Labels'] = kmeans.labels_
#HK_district_dish_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

LD_grid_merged = LD_grid

# merge df_grouped with df_data to add latitude/longitude for each neighbourhood
LD_grid_merged = LD_grid_merged.join(LD_grid_dish_sorted.set_index('grid'), on='grid')
LD_grid_merged['Cluster Labels']=[int(val) if isinstance(val,numbers.Number) & (math.isnan(val)==False) else 0 for val in LD_grid_merged['Cluster Labels']]
LD_grid_merged_drop=LD_grid_merged.dropna()

In [13]:
LD_grid_merged_drop

,grid,longitude,latitude,1st Most poi Type,2nd Most poi Type,3rd Most poi Type,4th Most poi Type,5th Most poi Type,6th Most poi Type,Cluster Labels
2,"2,0",2.229214,48.885906,Retail,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,0
4,"4,0",2.232038,48.885906,Travel and Transportation,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,0
6,"6,0",2.234861,48.885906,Travel and Transportation,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,0
7,"7,0",2.236273,48.885906,Landmarks and Outdoors,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,2
12,"12,0",2.243333,48.885906,Community and Government,Retail,Arts and Entertainment,Business and_Professional Services,Education,Health and Medicine,0
...,...,...,...,...,...,...,...,...,...,...
247,"7,12",2.236273,48.897131,Education,Arts and Entertainment,Business and_Professional Services,Community and Government,Health and Medicine,Landmarks and Outdoors,0
251,"11,12",2.241921,48.897131,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,3
252,"12,12",2.243333,48.897131,Business and_Professional Services,Arts and Entertainment,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,1
256,"16,12",2.248980,48.897131,Health and Medicine,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Landmarks and Outdoors,0


In [14]:
LD_grid_merged_drop.to_csv('LD_grid_ALLMEDIA_cluster.csv', index=False, encoding='utf-8')
LD_grid_merged_drop = pd.read_csv("LD_grid_ALLMEDIA_cluster.csv",engine='python',sep=',')
print(LD_grid_merged_drop)

      grid  longitude   latitude                   1st Most poi Type  \
0      2,0   2.229214  48.885906                              Retail   
1      4,0   2.232038  48.885906           Travel and Transportation   
2      6,0   2.234861  48.885906           Travel and Transportation   
3      7,0   2.236273  48.885906              Landmarks and Outdoors   
4     12,0   2.243333  48.885906            Community and Government   
..     ...        ...        ...                                 ...   
132   7,12   2.236273  48.897131                           Education   
133  11,12   2.241921  48.897131              Arts and Entertainment   
134  12,12   2.243333  48.897131  Business and_Professional Services   
135  16,12   2.248980  48.897131                 Health and Medicine   
136  19,12   2.253216  48.897131              Arts and Entertainment   

                      2nd Most poi Type                   3rd Most poi Type  \
0                Arts and Entertainment  Business and_Pr

In [15]:
# create map
map_cluster = folium.Map(tiles='cartodbpositron', location=[48.89114492, 2.240388888], zoom_start=14.6)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(LD_grid_merged_drop['latitude'], LD_grid_merged_drop['longitude'], LD_grid_merged_drop['grid'], LD_grid_merged_drop['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_cluster)
       
map_cluster.save('ld_district_ALLMEDIA.html')
display(IFrame('ld_district_ALLMEDIA.html', width = 1500, height = 1200))
print([cluster-1])

[2]


In [35]:
LD_grid_merged_drop[LD_grid_merged_drop['Cluster Labels'] == 0]

,grid,longitude,latitude,1st Most poi Type,2nd Most poi Type,3rd Most poi Type,4th Most poi Type,5th Most poi Type,6th Most poi Type,Cluster Labels
0,"2,0",2.229214,48.885906,Retail,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,0
1,"4,0",2.232038,48.885906,Travel and Transportation,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,0
2,"6,0",2.234861,48.885906,Travel and Transportation,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,0
4,"12,0",2.243333,48.885906,Community and Government,Retail,Arts and Entertainment,Business and_Professional Services,Education,Health and Medicine,0
5,"19,0",2.253216,48.885906,Community and Government,Arts and Entertainment,Business and_Professional Services,Education,Health and Medicine,Landmarks and Outdoors,0
8,"7,1",2.236273,48.886841,Retail,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,0
10,"11,1",2.241921,48.886841,Community and Government,Arts and Entertainment,Business and_Professional Services,Education,Health and Medicine,Landmarks and Outdoors,0
11,"12,1",2.243333,48.886841,Retail,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,0
13,"15,1",2.247568,48.886841,Travel and Transportation,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,0
16,"0,2",2.226390,48.887777,Community and Government,Arts and Entertainment,Business and_Professional Services,Education,Health and Medicine,Landmarks and Outdoors,0


In [36]:
LD_grid_merged_drop[LD_grid_merged_drop['Cluster Labels'] == 1]

,grid,longitude,latitude,1st Most poi Type,2nd Most poi Type,3rd Most poi Type,4th Most poi Type,5th Most poi Type,6th Most poi Type,Cluster Labels
9,"10,1",2.240509,48.886841,Business and_Professional Services,Retail,Arts and Entertainment,Community and Government,Education,Health and Medicine,1
18,"5,2",2.233449,48.887777,Business and_Professional Services,Arts and Entertainment,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,1
24,"16,2",2.248980,48.887777,Business and_Professional Services,Landmarks and Outdoors,Arts and Entertainment,Community and Government,Education,Health and Medicine,1
28,"1,3",2.227802,48.888712,Business and_Professional Services,Arts and Entertainment,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,1
30,"7,3",2.236273,48.888712,Business and_Professional Services,Arts and Entertainment,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,1
31,"8,3",2.237685,48.888712,Business and_Professional Services,Arts and Entertainment,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,1
33,"12,3",2.243333,48.888712,Business and_Professional Services,Arts and Entertainment,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,1
38,"1,4",2.227802,48.889648,Business and_Professional Services,Arts and Entertainment,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,1
39,"2,4",2.229214,48.889648,Business and_Professional Services,Arts and Entertainment,Travel and Transportation,Community and Government,Education,Health and Medicine,1
40,"3,4",2.230626,48.889648,Business and_Professional Services,Arts and Entertainment,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,1


In [37]:
LD_grid_merged_drop[LD_grid_merged_drop['Cluster Labels'] == 2]

,grid,longitude,latitude,1st Most poi Type,2nd Most poi Type,3rd Most poi Type,4th Most poi Type,5th Most poi Type,6th Most poi Type,Cluster Labels
3,"7,0",2.236273,48.885906,Landmarks and Outdoors,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,2
6,"2,1",2.229214,48.886841,Community and Government,Landmarks and Outdoors,Arts and Entertainment,Business and_Professional Services,Education,Health and Medicine,2
14,"17,1",2.250392,48.886841,Landmarks and Outdoors,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,2
15,"19,1",2.253216,48.886841,Community and Government,Landmarks and Outdoors,Arts and Entertainment,Business and_Professional Services,Education,Health and Medicine,2
17,"3,2",2.230626,48.887777,Landmarks and Outdoors,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,2
25,"17,2",2.250392,48.887777,Landmarks and Outdoors,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,2
26,"18,2",2.251804,48.887777,Landmarks and Outdoors,Travel and Transportation,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,2
29,"6,3",2.234861,48.888712,Arts and Entertainment,Landmarks and Outdoors,Business and_Professional Services,Community and Government,Education,Health and Medicine,2
34,"14,3",2.246157,48.888712,Landmarks and Outdoors,Retail,Community and Government,Arts and Entertainment,Business and_Professional Services,Education,2
35,"15,3",2.247568,48.888712,Landmarks and Outdoors,Retail,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,2


In [38]:
LD_grid_merged_drop[LD_grid_merged_drop['Cluster Labels'] == 3]

,grid,longitude,latitude,1st Most poi Type,2nd Most poi Type,3rd Most poi Type,4th Most poi Type,5th Most poi Type,6th Most poi Type,Cluster Labels
7,"4,1",2.232038,48.886841,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,3
12,"14,1",2.246157,48.886841,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,3
20,"9,2",2.239097,48.887777,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,3
21,"10,2",2.240509,48.887777,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,3
23,"15,2",2.247568,48.887777,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,3
51,"19,4",2.253216,48.889648,Arts and Entertainment,Education,Business and_Professional Services,Community and Government,Health and Medicine,Landmarks and Outdoors,3
84,"0,8",2.226390,48.893389,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,3
85,"3,8",2.230626,48.893389,Arts and Entertainment,Landmarks and Outdoors,Business and_Professional Services,Community and Government,Education,Health and Medicine,3
91,"9,8",2.239097,48.893389,Arts and Entertainment,Retail,Community and Government,Education,Landmarks and Outdoors,Business and_Professional Services,3
99,"7,9",2.236273,48.894325,Arts and Entertainment,Business and_Professional Services,Community and Government,Education,Health and Medicine,Landmarks and Outdoors,3
